In [10]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
from werkzeug.utils import secure_filename
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import socket

In [11]:
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})  # Enable CORS with more permissive settings

# Configuration
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}
MODEL_PATH = 'final-year-project2-model.keras'

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB max file size

# Create uploads folder if it doesn't exist
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

In [12]:
# Get local IP address
def get_local_ip():
    try:
        # Create a socket connection to get the local IP
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        local_ip = s.getsockname()[0]
        # print(local_ip)
        s.close()
        return local_ip
    except Exception:
        return "127.0.0.1"  # Fallback to localhost

In [13]:
class WasteClassifier:
    def __init__(self, weights_path):
        self.categories = ['can', 'glass', 'paper', 'plastic', 'plastic_bag', 'styrofoam']

        # Create model architecture
        base_model = VGG16(weights='imagenet',
                          include_top=False,
                          input_shape=(224, 224, 3))

        for layer in base_model.layers:
            layer.trainable = False

        model = Sequential([
            base_model,
            Flatten(),
            Dense(128, kernel_regularizer=l2(0.01)),
            Activation('relu'),
            Dropout(0.6),
            Dense(len(self.categories), activation='softmax')
        ])

        optimizer = Adam(learning_rate=0.0001)
        model.compile(
            loss='sparse_categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy']
        )

        model.build((None, 224, 224, 3))
        model.load_weights(weights_path)
        self.model = model

    def preprocess_image(self, image_path):
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Could not load image at {image_path}")

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        img = img.astype(np.float32) / 255.0
        img = np.expand_dims(img, axis=0)
        return img

    def predict_image(self, image_path):
        processed_image = self.preprocess_image(image_path)
        predictions = self.model.predict(processed_image, verbose=0)

        predicted_class_idx = np.argmax(predictions[0])
        confidence = float(predictions[0][predicted_class_idx])

        return {
            'class': self.categories[predicted_class_idx],
            'confidence': confidence,
            'predictions': {
                category: float(conf)
                for category, conf in zip(self.categories, predictions[0])
            }
        }


In [14]:
# Initialize the classifier
try:
    classifier = WasteClassifier(MODEL_PATH)
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {str(e)}")
    classifier = None

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({'status': 'healthy', 'model_loaded': classifier is not None})

@app.route('/predict', methods=['POST'])
def predict():
    if classifier is None:
        return jsonify({'success': False, 'error': 'Model not initialized'}), 503

    if 'image' not in request.files:
        return jsonify({'success': False, 'error': 'No image file provided'}), 400

    file = request.files['image']
    if file.filename == '':
        return jsonify({'success': False, 'error': 'No selected file'}), 400

    if file and allowed_file(file.filename):
        try:
            # Save the file temporarily
            filename = secure_filename(file.filename)
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)

            # Make prediction
            result = classifier.predict_image(filepath)

            # Clean up
            os.remove(filepath)

            return jsonify({
                'success': True,
                'prediction': result
            })

        except Exception as e:
            # Log the error for debugging
            print(f"Prediction error: {str(e)}")
            return jsonify({
                'success': False,
                'error': str(e)
            }), 500

    return jsonify({'success': False, 'error': 'Invalid file type'}), 400

Model loaded successfully!


In [17]:
if __name__ == '__main__':
    local_ip = get_local_ip()
    print(f"Server running on: http://{local_ip}:5000")
    app.run(host='0.0.0.0', port=5000, debug=False, threaded=True)

Server running on: http://172.0.1.87:5000
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.0.1.87:5000
Press CTRL+C to quit
172.0.2.89 - - [02/Dec/2024 12:58:13] "GET /health HTTP/1.1" 200 -
172.0.2.89 - - [02/Dec/2024 12:59:55] "POST /predict HTTP/1.1" 400 -
172.0.2.89 - - [02/Dec/2024 13:01:22] "POST /predict HTTP/1.1" 200 -
172.0.2.89 - - [02/Dec/2024 13:01:45] "POST /predict HTTP/1.1" 400 -
